In [2]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import smart_open
import numpy as np 
from sklearn import preprocessing

In [3]:
data1 = open('./data/train_data.word.ed',encoding='utf-8').readlines()
data2 = open('./data/val_data.word.ed',encoding='utf-8').readlines()

In [4]:
split_data1 = [i.split(r'\t') for i  in data1]
labels1 = [i[0] for i in split_data1]
text1 = [i[1] for i in split_data1]

split_data2 = [i.split(r'\t') for i  in data2]
labels2 = [i[0] for i in split_data2]
text2 = [i[1] for i in split_data2]

In [5]:
train_df = pd.DataFrame([i,j] for i,j in zip(labels1,text1))
eval_df = pd.DataFrame([i,j] for i,j in zip(labels2,text2))

In [6]:
train_df[0][:10]

0          society
1            world
2    entertainment
3              car
4             baby
5           sports
6             tech
7          society
8         military
9    entertainment
Name: 0, dtype: object

In [7]:
le = preprocessing.LabelEncoder() 
le.fit(np.unique(train_df[0].tolist()))
print('标签值标准化:%s' % le.transform(["world", "entertainment", "car", "baby","entertainment"]))
print('标准化标签值反转:%s' % le.inverse_transform([0, 2 ,0 ,1 ,2]))

标签值标准化:[17  3  1  0  3]
标准化标签值反转:['baby' 'discovery' 'baby' 'car' 'discovery']


In [8]:
train_df[2] = train_df[0].apply(lambda x:le.transform([x])[0])
eval_df[2] = eval_df[0].apply(lambda x:le.transform([x])[0])

del train_df[0]
del eval_df[0]

In [9]:
num_labels =len(np.unique(train_df[2].tolist()))
num_labels

18

In [10]:
train_df.to_csv('train_data.csv',header=False,sep='\t',index = False)
eval_df.to_csv('test_data.csv',header=False,sep='\t',index = False)

In [11]:
train_df.head()

,1,2
0,陆家嘴 不 雅 视频 曝光 ： 系恶 “ 性 ” 营销 ？\n,12
1,日本 冲绳 民众 举行 大规模 抗美 集会 活动 冲绳 县知事 : 希望 改变 政治 结构\n,17
2,林心如 全黑 连体 装 现身 ， 被 赞比 模特 身材 更好\n,3
3,别克 逆 生长 实验 型\n,1
4,你 的 宝宝 到底 需不需要 补锌 ？ 又 该 如何 补充 ？ （ 内附 补锌 食谱 ）\n,0


In [12]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47784 entries, 0 to 47783
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       47784 non-null  object
 1   2       47784 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 746.8+ KB


In [13]:
# model = ClassificationModel(
#     "bert", "./data/chinese_wwm_ext_pytorch",
#     num_labels=num_labels, 
#     args={"reprocess_input_data": True,   # 对输入数据进行预处理
#           "overwrite_output_dir": True}   # 可覆盖输出文件夹
#     )
model = ClassificationModel("bert", "./outputs/checkpoint-41812-epoch-1")

In [14]:
# 开始模型训练
# 许多应用都会得益于使用半精度来储存数据，然后用32位的单精度来处理这些数据。高级的GPU将会全面支持这种“混合精度”的计算，使用FP16计算将会获得比FP32和FP64更高的吞吐量，提高训练速度，但是在windows的环境下很难安装。。。
model.train_model(train_df,args = {'fp16':False})

Epoch: 100%|██████████| 1/1 [00:00<00:00, 501.29it/s] 


(41812, 0.0)

In [15]:
# 评测模型
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score,
                                                            #f1 = sklearn.metrics.f1_score
                                                           )

Running Evaluation: 100%|██████████| 5973/5973 [04:08<00:00, 24.05it/s]


In [16]:
print(result)

{'mcc': 0.8366276825210569, 'acc': 0.8527540599363804, 'eval_loss': 0.4950253406088009}


In [19]:
predictions, raw_outputs = model.predict(["卡舒吉案“最高机密”文件披露，杀手与沙特王储有关 ",
                                            "农心杯申真谞屠龙豪取四连胜 柯洁明日登场攻擂",
                                            "CentOS 退出江湖，服务器操作系统该选择 Debian 还是 Ubuntu？",
                                            "比特币带“疯”电脑显卡！商家：找我买卡的十有六七为挖矿",
                                            "一白遮百丑？咖啡系女明星请把身体还给自己",
                                            "高血压,冠状动脉粥样硬化性心脏病,高脂血症",
                                            "新能源汽车人才有机会落户上海"
                                          ])

100%|██████████| 1/1 [00:00<00:00, 16.99it/s]


In [21]:
labels = le.inverse_transform(predictions)
labels 

array(['world', 'sports', 'tech', 'tech', 'entertainment', 'regimen',
       'car'], dtype='<U13')